# Activity Chain Optimization - exact solution with modified Held-Karp

In [3]:
import numpy as np
from typing import NamedTuple
import itertools as it

In [301]:
class SolutionLevel(NamedTuple):
    # mandatory
    level: int                        # 0 to m-1 incl.: how many non-home services are included
    nof_sols: int                     # bounded by O(m_Choose_level * n)
    configs: np.ndarray               # (nof_sols, m+1): l selected services out of m [T/F] and 1 destn. [citynum]
    paths: np.ndarray                 # (nof_sols, n): each city has a sequence nr if it lies on the path, or 0
    costs: np.ndarray                 # (nof_sols,)

class ActivityChainProblem:
    def __init__(self, tsp_graph, service_constraints=None, cost_at_arrival_constraints=None,
                 start_cost=420, max_cost=1440):     # start at 7am and travel until 12pm at most
        # Problem definition
        self.graph = tsp_graph                       # (n,n): Asymmetric distances between cities (0 is home)
        self.services = service_constraints          # (n,m): Boolean: does city n offer service m? (svc 0 is home)
        self.arrivals = cost_at_arrival_constraints  # (n,2): [from_cost, to_cost]
        if self.services is None:
            self.services = np.identity(self.graph.shape[0])
        if self.arrivals is None:
            self.arrivals = np.zeros((self.graph.shape[0],2))
            self.arrivals[:, 0] = start_cost
            self.arrivals[:, 1] = max_cost
        self.n = self.graph.shape[0]                 # nof cities
        self.m = self.services.shape[1]              # nof services
        self.start_cost = start_cost
        self.max_cost = max_cost
        assert self.graph.shape[0] == self.graph.shape[1]
        assert self.graph.shape[0] == self.services.shape[0]
        assert self.graph.shape[0] == self.arrivals.shape[0]
        assert self.arrivals.shape[1] == 2
        assert (self.graph >= 0).all()
        assert (self.arrivals >= 0).all()
        assert (self.arrivals[:,0] <= self.arrivals[:,1]).all()
        assert self.start_cost <= self.max_cost
        assert (np.sum(self.services, axis=1) == 1).all()  # for now we assume each city offers exactly 1 service
        
    def solve(self):
        if self.n == 0 or self.m == 0:
            print("Cannot solve empty problem.")
            return None
        presol = self._presolution()
        costs = presol.costs + self.graph[presol.configs[:, -1], 0]  # add cost of finally going back home
        final_idxs = np.flatnonzero(costs == np.amin(costs))
        final_cost = costs[final_idxs[0]]
        final_paths = presol.paths.copy().astype(int)[final_idxs, :]
        sorted_idxs = np.argsort(final_paths)
        readable_paths = np.zeros((final_paths.shape[0], self.m+1)).astype(int)
        for i in range(readable_paths.shape[0]):
            k = 1
            for j in range(sorted_idxs.shape[1]):
                next_city_seqnum = final_paths[i, sorted_idxs[i,j]]
                if next_city_seqnum > 0:
                    readable_paths[i, k] = sorted_idxs[i,j]
                    k += 1
        if final_cost > self.max_cost:
            print(f"No solution found. Lowest cost {final_cost} exceeds limit {self.max_cost}.")
            return None
        return readable_paths, final_cost
        
    def _presolution(self):
        """Modified Held-Karp algorithm.
        Grows solutions starting with 1 non-home svc up to and including m-1 non-home services."""
        prev_level = None
        next_level = self._zeroth_solution()
        for i in range(1, self.m):
            prev_level = next_level
            next_level = self._next_solution(prev_level)
            print(f"Solved Level {next_level.level}: \
                  {next_level.nof_sols} partial solutions, \
                  minimum cost: {np.amin(next_level.costs)}")
        return next_level
    
    def _zeroth_solution(self):
        configs = np.zeros((1, self.m+1))
        configs[0, 0] = 1
        return SolutionLevel(level=0, nof_sols=1,
                             configs=np.array(configs, dtype=np.int32),
                             paths=np.zeros((1, self.n), dtype=np.int32),
                             costs=np.array([self.start_cost]))
    
    def _next_solution(self, prev):
        level = prev.level + 1
        nonhome_svcs = np.arange(1, self.m)
        nonhome_dests = np.arange(1, self.n)
        svc_combs = it.combinations(nonhome_svcs, level)
        configs, paths, costs = None, None, None
        for svc_comb in svc_combs:
            svc_comb = np.asarray(svc_comb)
            svc_comb_flags = np.zeros(self.m).astype(int)
            svc_comb_flags[svc_comb] += 1
            svc_comb_flags[0] += 1  # include home service as visited
            # level == svcnum-1 == nof_svcs_intersecting_with_ancestors_among_previous
            ancestor_flags = (level-np.sum(svc_comb_flags * prev.configs[:,:-1], axis=1) == 0)
            ancestor_idxs = np.flatnonzero(ancestor_flags)
            ancestor_configs = prev.configs[ancestor_idxs]
            eligible_dests = np.sum(self.services[:, svc_comb], axis=1)
            eligible_dests[0] = 0  # exclude home from destinations
            eligible_dests = np.transpose(np.nonzero(eligible_dests))
            svc_comb_block = svc_comb_flags * np.ones((eligible_dests.shape[0], svc_comb_flags.shape[0]))
            configs_chunk = np.append(svc_comb_block, eligible_dests, axis=1).astype(int)
            paths_chunk = np.zeros((configs_chunk.shape[0], self.n)).astype(int)
            costs_chunk = np.ones(configs_chunk.shape[0]) * np.finfo(np.float32).max
            for i in range(configs_chunk.shape[0]):
                dest_city = configs_chunk[i, -1]
                dest_svc = np.flatnonzero(self.services[dest_city])[0]  # first (and only) svc offered in city
                parent_idxs = np.flatnonzero(ancestor_configs[:, dest_svc] == 0)
                parent_dests = ancestor_configs[parent_idxs, -1]
                parent_costs = prev.costs[ancestor_idxs[parent_idxs]]
                laststep_costs = self.graph[parent_dests, dest_city]
                total_parent_costs = parent_costs + laststep_costs
                early_parents = total_parent_costs < self.arrivals[dest_city, 0]
                total_parent_costs[early_parents] = self.arrivals[dest_city, 0]
                late_parents = total_parent_costs > self.arrivals[dest_city, 1]
                total_parent_costs[late_parents] = self.max_cost
                best_parent_idx = np.argmin(total_parent_costs)  # TODO: here we forget other equally cheap paths
                best_parent_cost = total_parent_costs[best_parent_idx]
                best_parent_path = prev.paths[ancestor_idxs[parent_idxs[best_parent_idx]]]
                costs_chunk[i] = best_parent_cost
                paths_chunk[i] = best_parent_path.copy()
                paths_chunk[i, dest_city] = level  # attach dest city to path (level-th)
            if configs is None:
                configs = configs_chunk
                paths = paths_chunk
                costs = costs_chunk
            else:
                configs = np.append(configs, configs_chunk, axis=0)
                paths = np.append(paths, paths_chunk, axis=0)
                costs = np.append(costs, costs_chunk, axis=0)
        # print(f"configs:\n{configs}")
        # print(f"paths:\n{paths}")
        # print(f"costs:\n{costs}")
        nof_sols=configs.shape[0]
        return SolutionLevel(level, nof_sols, configs, paths, costs)


In [302]:
def test(aco):
    print(f"Solving for {aco.m} services in {aco.n} cities:")
    fin_paths, fin_cost = aco.solve()
    print(f"\nFound at least {fin_paths.shape[0]} optimal path(s) at cost {fin_cost}")
    for path in fin_paths:
        print(f"\t>> {path}")

In [303]:
def generate_dummy_01():
    tsp_graph = np.array([[0, 5, 3, 2],
                          [5, 0, 4, 1],
                          [3, 4, 0, 2],
                          [2, 1, 2, 0]])
    service_constraints=None
    cost_at_arrival_constraints=None
    return ActivityChainProblem(tsp_graph, service_constraints, cost_at_arrival_constraints)

In [304]:
def generate_dummy_02(size):
    not_identity = -1 * (np.identity(size)-1)
    tsp_graph = np.random.rand(size, size) * not_identity
    service_constraints=None
    cost_at_arrival_constraints=None
    return ActivityChainProblem(tsp_graph, service_constraints, cost_at_arrival_constraints)

In [305]:
def generate_dummy_03(size, nof_nonhome_svcs):
    assert 0 < nof_nonhome_svcs < size
    not_identity = -1 * (np.identity(size)-1)
    tsp_graph = np.random.rand(size, size) * not_identity
    service_constraints = np.zeros((size, nof_nonhome_svcs+1))
    service_constraints[0, 0] = 1
    district_borders = np.random.choice(np.arange(2, size), nof_nonhome_svcs-1, replace=False)
    district_borders.sort()
    for i in range(district_borders.shape[0]):
        if i == 0:  # if first border
            service_constraints[1:district_borders[i], i+1] = 1
        if i == district_borders.shape[0]-1:  # if last border
            service_constraints[district_borders[i]:, i+2] = 1
        else:
            service_constraints[district_borders[i]:district_borders[i+1], i+2] = 1
    cost_at_arrival_constraints=None
    return ActivityChainProblem(tsp_graph, service_constraints, cost_at_arrival_constraints)

In [313]:
def generate_dummy_04(size, nof_nonhome_svcs):
    assert 0 < nof_nonhome_svcs < size
    not_identity = -1 * (np.identity(size)-1)
    tsp_graph = np.random.rand(size, size) * not_identity
    service_constraints = np.zeros((size, nof_nonhome_svcs+1))
    service_constraints[0, 0] = 1
    district_borders = np.random.choice(np.arange(2, size), nof_nonhome_svcs-1, replace=False)
    district_borders.sort()
    for i in range(district_borders.shape[0]):
        if i == 0:  # if first border
            service_constraints[1:district_borders[i], i+1] = 1
        if i == district_borders.shape[0]-1:  # if last border
            service_constraints[district_borders[i]:, i+2] = 1
        else:
            service_constraints[district_borders[i]:district_borders[i+1], i+2] = 1
    cost_at_arrival_constraints = np.random.choice(np.arange(420, 1440), size*2).reshape(size, 2)
    cost_at_arrival_constraints.sort(axis=1)
    return ActivityChainProblem(tsp_graph, service_constraints, cost_at_arrival_constraints)

In [315]:
aco = generate_dummy_01()
%time test(aco)

Solving for 4 services in 4 cities:
Solved Level 1:                   3 partial solutions,                   minimum cost: 422.0
Solved Level 2:                   6 partial solutions,                   minimum cost: 423.0
Solved Level 3:                   3 partial solutions,                   minimum cost: 426.0

Found at least 2 optimal path(s) at cost 430.0
	>> [0 3 1 2 0]
	>> [0 2 1 3 0]
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.84 ms


In [316]:
aco = generate_dummy_02(14)
%time test(aco)

Solving for 14 services in 14 cities:
Solved Level 1:                   13 partial solutions,                   minimum cost: 420.04617543816283
Solved Level 2:                   156 partial solutions,                   minimum cost: 420.08759553227685
Solved Level 3:                   858 partial solutions,                   minimum cost: 420.0959653886801
Solved Level 4:                   2860 partial solutions,                   minimum cost: 420.1070149938053
Solved Level 5:                   6435 partial solutions,                   minimum cost: 420.20200847599307
Solved Level 6:                   10296 partial solutions,                   minimum cost: 420.2291785174202
Solved Level 7:                   12012 partial solutions,                   minimum cost: 420.239621075936
Solved Level 8:                   10296 partial solutions,                   minimum cost: 420.342860487175
Solved Level 9:                   6435 partial solutions,                   minimum cost: 420.4457

In [317]:
aco = generate_dummy_03(1000, 7)
%time test(aco)

Solving for 8 services in 1000 cities:
Solved Level 1:                   999 partial solutions,                   minimum cost: 420.0015943258866
Solved Level 2:                   5994 partial solutions,                   minimum cost: 420.00184291790265
Solved Level 3:                   14985 partial solutions,                   minimum cost: 420.0040019213533
Solved Level 4:                   19980 partial solutions,                   minimum cost: 420.0052691042377
Solved Level 5:                   14985 partial solutions,                   minimum cost: 420.007295947574
Solved Level 6:                   5994 partial solutions,                   minimum cost: 420.0125463004791
Solved Level 7:                   999 partial solutions,                   minimum cost: 420.01744043845565

Found at least 1 optimal path(s) at cost 420.03144933948346
	>> [  0 583 564 485 996 151  73 935   0]
CPU times: user 10.1 s, sys: 2.41 s, total: 12.5 s
Wall time: 3.2 s


In [320]:
aco = generate_dummy_04(1000, 7)
%time test(aco)

Solving for 8 services in 1000 cities:
Solved Level 1:                   999 partial solutions,                   minimum cost: 420.19317404184756
Solved Level 2:                   5994 partial solutions,                   minimum cost: 420.2832182637197
Solved Level 3:                   14985 partial solutions,                   minimum cost: 421.0
Solved Level 4:                   19980 partial solutions,                   minimum cost: 421.022089320173
Solved Level 5:                   14985 partial solutions,                   minimum cost: 421.13849606292916
Solved Level 6:                   5994 partial solutions,                   minimum cost: 424.0
Solved Level 7:                   999 partial solutions,                   minimum cost: 453.0

Found at least 1 optimal path(s) at cost 453.1408885003228
	>> [  0 735 414 688  95  54 899 316   0]
CPU times: user 11.3 s, sys: 2.14 s, total: 13.4 s
Wall time: 3.46 s
